# 一些思考问题

---

突然想发现一个问题, 为什么使用JS的时候很少导入包呢?

1. DOM Interface是一个, 其实也不算, 毕竟DOM不是JS语言的一部分, 但是也没有导入其他包啊, 怎么直接就用的?

2. Map, Set, Math算法等, 这都是JS语言实现的, 那为什么也不需要导入包呢? 你看CPP的STL, Java的Util, 使用容器啥的都需要导入. 这样实现的好处在哪呢?

3. 再进一步想, HTML/CSS也是一种语言, 那么HTML/CSS有没有编译器呢? 

## 问题二
第二个问题的前半部分的答案已经找到, 原来map,Math, 都是JS的built-in对象:
> Built-in objects and functions, including JSON, Math, Array methods, parseInt, decodeURI, etc. [Find More](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects)

## 问题一
关于第一个问题, 我看到了官方的这句话:

You don't have to do anything special to begin using the DOM. You use the API directly in JavaScript from within what is called a script, a program run by a browser. 这就是说不需要额外导入什么包, 拿到js就能用, 给人的感觉好像是DOM嵌入了JS. 但真实是这样吗? 是不是JS engine实现了DOM呢?还是Browser里面的其他程序实现了DOM? 想继续探讨 还需要先看看第三个问题.

## 问题三
第三个问题, HTML是markup language, 并不是programming language. 所以并不需要Compiler. 看到一句话如下:

Unlike programming languages, markup languages are not executed: they are read and rendered instead. 也就是说浏览器里面有个Render去帮我们渲染显示HTML source文件. 

## 浏览器组成

这些问题越思考越是晕, DOM在哪实现的? JS的内置对象是怎么实现的? HTML是怎么被渲染的? 好像都和浏览器有关系, 不如 直接去探索下浏览器的组成.. 只列出几个部分, 怪不得说自己实现一个浏览器是不可能的....

1. The rendering engine

responsible for displaying requested content. For example if the requested content is HTML, the rendering engine parses HTML and CSS, and displays the parsed content on the screen.

2. JavaScript interpreter

Used to parse and execute JavaScript code.

3. Networking

For network calls such as HTTP requests, using different implementations for different platform behind a platform-independent interface.


从上面可以看出, 浏览器里有两个大模块：rendering engine（比如 blink，webkit），和 JS 引擎（比如 V8）。前者负责 html 解析和渲染，后者负责执行 JS。 看到这还是有很多疑惑,还是不知道DOM在哪被实现? 接下来就看看网页rendering的过程,希望能从中找到想要的答案. 

https://www.ques10.com/p/19744/what-are-the-major-components-of-a-web-browser-d-1/?


## 关于第一个问题

在看网页渲染过程之前插播一段关于第一个问题的疑问. DOM里面的Interfaces并不是Javascript的对象, 因为官方文件就说了, DOM API并不属于JS语言本身, 所以DOM Interfaces并不是built-in objects. 但是, 通过实现, 在runtime的时候, DOM Interfaces表现的行为就 JS的 built-in objects. 那么DOM是谁提供的呢? 答: Browser

> Strictly speaking, no. The JavaScript runtime has access to them, and in that capacity they can function as JavaScript objects. But they are defined in a way that is not bound to any particular language, and in most DOM implementations, they're native code.        
For most practical purposes, when you're working in the browser or in some other JavaScript runtime, yes. As I stated above, most DOM implementations try to make the DOM objects work the same way as other objects in the language, and for JavaScript, that means making them work like "real" JavaScript objects.

https://stackoverflow.com/a/32844463/16317008

[Built-in Objects in Javascript](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects)

JavaScript doesn’t understand what DOM is, it is not part of the JavaScript specifications. DOM is a high-level Web API **provided by the browser**. 


# 网页渲染的过程

---

找到[一篇文章](https://medium.com/jspoint/how-the-browser-renders-a-web-page-dom-cssom-and-rendering-df10531c9969), 看着写的超级棒, 先读读(excited)...

建立了很多完整的概念, 尤其是渲染的过程, 从获得服务器发送的binary内容到解析...


### 渲染过程总结为两段话

When a web page is loaded, the browser first reads the `HTML text` and constructs `DOM Tree` from it. Then it processes the CSS whether that is inline, embedded, or external CSS and constructs the `CSSOM Tree` from it.

After these trees are constructed, then it constructs the `Render-Tree` from it. Once the Render-Tree is constructed, then the browser starts the printing individual elements on the screen.

The job of creating DOM Tree, CSSOM Tree, and handle rendering logic is done using a piece of software called a `Browser Engine` (also known as Rendering Engine or Layout Engine) which resides inside the browser. 这个在文章上面浏览器组成那节已经介绍, 可以去看看. 

#### 当客户端获得到服务器响应的binary文件

```html
Content-Type: text/html; charset=UTF-8
```
Here `text/html` is a MIME Type which tells the browser that it is an HTML document and `charset=UTF-8` tells the browser that it is encoded in UTF-8 character encoding. Using this information, the browser can convert the binary format into a readable text file. If this header is missing, the browser would not understand how to process the file and it will render in plain text format. 

But if everything is OK, after this conversion, the browser can start reading the HTML document. 

#### 当把binary文件解码成可读文件后 浏览器开始为每个element都创造一个对应的node, 例如comment(注释)也是一个node, 只是type不同

When the **browser** reads HTML code, whenever it encounters an HTML element like `html`, `body`, `div` etc., **it(browser)** creates a JavaScript object called a `Node`. Eventually, all HTML elements will **be converted to** JavaScript objects.

#### 浏览器convert好全部node后, 就会创建 node tree 这样就方便遍历管理每一个节点啦

After the browser has created Nodes from the HTML document, it has to create a tree-like structure of these node objects.

> A DOM node doesn’t always have to be an HTML element. When the browser creates a DOM tree, it also saves things like `comments`, `text` as separate nodes in the tree. 


> JavaScript doesn’t understand what DOM is, it is not part of the JavaScript specifications. DOM is a high-level Web API provided by the browser. 

#### 创建好DOM tree后 便开始创建 CSS Tree

After constructing the DOM, the browser reads CSS from all the sources (external, embedded, inline, user-agent, etc.) and construct a CSSOM(CSS Object Model). CSSOM stands for CSS Object Model which is a Tree Like structure just like DOM. CSS tree结构和DOM完全一样,只是每个Node下面多了一些属性, 比如color, width.

DOM tree是框架, 那CSS tree就是细节.

#### 创建完CSS Tree 接着创建Render Tree

Render-Tree is also a tree-like structure constructed by combining DOM and CSSOM trees together.

文章后续还介绍了js解析等过程, 具体不在这展开, 感兴趣可以去看看

https://medium.com/jspoint/how-the-browser-renders-a-web-page-dom-cssom-and-rendering-df10531c9969

# DOM

---

The Document Object Model (DOM) **connects** web pages **to** scripts or programming languages **by** representing the structure of a document(such as the HTML) in memory. 

A web page is a document that can be either displayed in the browser window or as the HTML source. In both cases, it is the same document but the Document Object Model (DOM) representation allows it to be manipulated. 

How to understand this paragraph below:

> The DOM is not part of the JavaScript language, but is instead a Web API used to build websites. JavaScript can also be used in other contexts. For example, `Node.js` runs JavaScript programs on a computer, but provides a different set of APIs, and the DOM API is not a core part of the Node.js runtime.


